In [ ]:
# from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import wordnet as wn
import random
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from PyDictionary import PyDictionary

def tag(sentence):
    words = word_tokenize(sentence)
    words = pos_tag(words)
    #print(words)
    return words
    

def paraphraseable(tag):
    return tag.startswith('NN') or tag == 'VB' or tag.startswith('JJ')

def pos(tag):
    if tag.startswith('NN'):
        return wn.NOUN
    elif tag.startswith('V'):
        return wn.VERB
    elif tag.startswith('DT'):
        return 'that'
    elif tag.startswith('JJ'):
        return wn.VERB
    elif tag.startswith('IN'):
        return wn.ADJECTIVE
    elif tag.startswith('VBZ'):
        return wn.VERB

def synonyms(word, tag):
    lemma_lists = [ss.lemmas() for ss in wn.synsets(word, pos(tag))]
    lemmas = [lemma.name() for lemma in sum(lemma_lists, [])]
    #print(set(lemmas))
    return set(lemmas)

def synonymIfExists(sentence):
    for (word, t) in tag(sentence):
        if paraphraseable(t):
            syns = synonyms(word, t)
            syns=list(syns)
            if len(syns)>1:
                #print(syns)
                secure_random = random.SystemRandom()
                k=secure_random.choice(syns)
                #print(k)
            if syns:
                if len(syns)>1:
                    yield [k]
                    continue
        yield [word]


def paraphrase(sentence):
    return [x for x in synonymIfExists(sentence)]

def definition(str1):
    dictionary=PyDictionary(str1)
    return dictionary.getMeanings() 
def antonyms(str2):
    synonyms = [] 
    antonyms = [] 
    for syn in wordnet.synsets(str2): 
        for l in syn.lemmas(): 
            synonyms.append(l.name()) 
            if l.antonyms(): 
                antonyms.append(l.antonyms()[0].name()) 
    #print(list(set(synonyms))) 
    return (list(set(antonyms)),list(set(synonyms)))
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
#Using Spacy and wordnet tool for implementing this idea.
#Trying to make it simple without going for complex neural networks
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from random import randint
import nltk.data
import pandas as pd
import spacy
import en_core_web_sm
from random_word import RandomWords
from nltk.stem.wordnet import WordNetLemmatizer
from PyDictionary import PyDictionary

dictionary=PyDictionary()
nlp = en_core_web_sm.load()
#nlp = spacy.load('C:\\Users\\DineshRajan\\Desktop\\DataSet\\en_core_web_sm-2.0.0\\en_core_web_sm\\en_core_web_sm-2.0.0')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
# Load a text file if required
dataset=pd.read_csv("C:\\Users\\DineshRajan\\Desktop\\DataSet\\Results.csv")
# dataset=pd.read_csv("E:\\dinesh\\valuelabs\\sam_csv.csv")
#print(dataset.head())
data=dataset['answer_text']
#print(dataset)
#print(data)
#h=data.head()

for q in range(len(data)):
    m_list=["January","Jan","February","Feb","March","Mar","April",
            "Apr","June","Jun","July","August","Aug","Sept","September",
            "October","Oct","November","Nov","December","Dec"]
    text = data[q]
    flag=0
    p=2
    k=text.split(" ")
    days=["monday","tuesday","wednesday","thursday","friday","saturday","sunday","Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday","Fridays" ]
    output=[]
    money=["Million","million","Billion","Billion","Millions","millions","Billions","Billions"]
    money1=["Millionaire","Billionaire","millionaire","billionaire"]
    number=["two","three","four","five","six","seven","nine","ten","Two","Three","Four","Five","Six","Seven","Nine","Ten","one","One"]
    #print(k)
    for i in range(len(k)):
#         if k[i] in m_list:
#             k[i]=random.choice(m_list)
            #print(k[i])
        if str(k[i]) in days:
                k[i]=random.choice(days)
#         if k[i].isdigit():
#                 w=int(k[i])
#                 w=w+3
#                 k[i]=w
    #print(k)
    m_str= ' '.join(map(str, k)) 
#     print(m_str)
#     print("above is string")QQ

    output=""

    tokenized = tokenizer.tokenize(text)
#     print(tokenized)

    # Get the list of words from the entire text
    words = word_tokenize(text)
#     print(words)

    # Identify the parts of speech
    tagged = nltk.pos_tag(words)
#     print(tagged)
    for j in range(p):
        opt=['\'']
        for i in range(0,len(words)):
            replacements = []

            for syn in wordnet.synsets(words[i]):

                # Do not attempt to replace proper nouns or determiners
                if tagged[i][1] == 'NNP' or tagged[i][1] == 'DT':
                    break
                word_type = tagged[i][1][0].lower()
                if syn.name().find("."+word_type+"."):
                    # extract the word only
                    r = syn.name()[0:syn.name().find(".")]
                    replacements.append(r)

            if len(replacements) > 0:
                # Choose a random replacement
                replacement = replacements[randint(0,len(replacements)-1)]
                output = output + " " + replacement
            else:
                # If no replacement could be found, then just use the
                # original word
                output = output + " " + words[i]
#         output=output+'\',\''  
        opt.append(output)
    colors=["blue","black","violet","red","white","green","yellow","pink","rose","orange","Orange","Blue","Black","Violet","Red","White","Green","Yellow","Pink","Rose"]
    doc = nlp(data[q])
    s = list(doc)
    tmp,temp,sub = "","",-1
    for i in doc:
        if i.pos_ == 'VERB':
            s[i.i] = i
        elif i.dep_ == 'nsubj':
            sub = i.i
            temp = i
        elif i.dep_ == 'dobj':
            tmp = i.text.capitalize()
            s[i.i] = temp
            s.insert(i.i,"by")

    s[sub] = tmp
    extra=' '.join(str(e) for e in s)
    output=list(opt)
    output.append(extra)
    new_list=[]
    #print(output)
    new_list.append('\'')
    new_list.append(output[1])
    #new_list.append(output[2])
    #print(new_list)
    names=['John','David','Joseph','Ashiq','Bala','Sathish','Praveen','Gopi','Arun','Gokul','Dinesh','Rajan','Oliver','Jaga']
    relations=["father","mother","uncle","aunt","brother","sister","cousin","Father","Mother","Uncle","Aunt","Brother","Sister"]
    metal=["gold","silver","Gold","Silver","bronze","Bronze"]
    if __name__ == '__main__':
        #h=data.head()
        text = data[q]
        a=paraphrase(text)
#     b=paraphrase1("The quick brown fox jumps over the lazy dog.")
#     print(b)
        s=' '.join(map(str,a))
        st=" "
        for i in range(len(s)):
            p=s[i]
            if(p!='[' and p!=']' and p!='\'' and p!='\''):
                st=st+p
        a1=paraphrase(text)
        s1=' '.join(map(str,a1))
        st1=" "
        for i in range(len(s1)):
            p1=s1[i]
            if(p1!='[' and p1!=']' and p1!='\'' and p1!='\''):
                st1=st1+p1
        #print(st)
        new_list.append(st)
        new_list.append(m_str)
        del new_list[0]
        finalize=[]
        g=new_list[1].split(" ")
        #print(g)
        for i in range(len(g)):
            if g[i] in m_list:
                g[i]=random.choice(m_list)
#             if g[i].isdigit():
#                 w=int(g[i])
#                 w=w+2
#                 g[i]=w
            if g[i] in relations:
                g[i]=random.choice(relations)
            if g[i] in number:
                g[i]=random.choice(number)
        new_string=" ".join(map(str,g))
        finalize.append(new_string)
        #finalize.append(new_list[1])#by
        #finalize.append(new_list[2].lstrip())#number
        g2=new_list[2].split(" ")
        #print(g)
        for i in range(len(g2)):
            if g2[i] in money:
                g2[i]=random.choice(money)
            if g2[i].isdigit():
                w=int(g2[i])
                w=w+2
                g2[i]=w
            if g2[i] in money1:
                g2[i]=random.choice(money1)
        new_string2=" ".join(map(str,g2))
        finalize.append(new_string2)
        single_word=[]
        #subs=["Maths","Physics","Mathematics","Biology","biology","science","Science","chemistry"]
        nu=["first","second","third","fouth","fifth","Fifth","sixth","seventh","tenth"]
        day1=["night","day","light","dark"]
        yer=["Years","years","year","days","hours","Days","hour","minute"]
        
        if len(g)<=2:
            
            sy,ay=antonyms(g[1])
            #print(sy)
            #print(ay)
            if len(sy):
                single_word.append(random.choice(sy))
#             if len(single_word)<2:
#                 single_word.append(g[1].lstrip())
            if len(ay):
                single_word.append(random.choice(ay))
#             if len(single_word)<2:
#                 single_word.append(g[1].lstrip())
            s=definition(g[1])
            e=s[g[1]]
            #print(e)
            e_str=str(e)
            
            if e_str =="None":#name and month check here
                sss=str(g[1])
                if(sss[0:3]) in m_list:
                    finalize.append(random.choice(m_list))
                    #single_word.append(random.choice(m_list))
                else:
                    finalize.append(random.choice(names))
                    single_word.append(str(g[1]).lstrip())
            else:
                hhh=e.get('Noun')
#                 print(hhh)
                if str(hhh)=="None":
                    ssh=e.get('Adjective')
                    if str(ssh)=="None":
                        qwe=e.get('Verb')
                        if str(qwe)=="None":
                            kkk=e.get('Adverb')
                            rep=" ".join(map(str,kkk))
                            finalize.append(rep[0:40])
                            single_word.append(rep[0:40])
                        else:
                            yur=" ".join(map(str,qwe))
                            finalize.append(yur[0:40])
                            single_word.append(yur[0:40])
                    else:
                        sey=" ".join(map(str,ssh))
                        finalize.append(sey[0:40])
                        single_word.append(sey[0:40])
         
                else:
                    yey=" ".join(map(str,hhh))
                    #print(yey)
                    finalize.append(yey[0:40])
                    single_word.append(yey[0:40])
        else:
            finalize.append(new_list[2].lstrip())
            single_word.append(new_list[2].lstrip())
        if len(single_word)<=2:
            single_word.append(str(g[1]).lstrip())
        one=finalize[0]
        tx=finalize[1]
        num_change=finalize[2]
        seasons=["summer","winter","autumn","spring","Summer","Winter","Autumn","Spring"]
        g3=num_change.split(" ")
        for i in range(len(g3)):
            if g3[i] in money:
                g3[i]=random.choice(money)
#             if g3[i] in colors:
#                 g3[i]=random.choice(colors)
            if g3[i] in seasons:
                g3[i]=random.choice(seasons)
            if g3[i] in number:
                g3[i]=random.choice(number)
            if g3[i] in yer:
                g3[i]=random.choice(yer)
            if g3[i].isdigit():
                w=int(g3[i])
                w=w+1
                g3[i]=w
            if g3[i] in m_list:
                g3[i]=random.choice(m_list)
        new_string4=" ".join(map(str,g3))
        tx_list=tx.split(" ")
        #print(tx_list)
        for i in range(len(tx_list)):
            if tx_list[i] in day1:
                tx_list[i]=random.choice(day1)
            if tx_list[i] in nu:
                tx_list[i]=random.choice(nu)
            if tx_list[i] in seasons:
                tx_list[i]=random.choice(seasons)
            if tx_list[i] in metal:
                tx_list[i]=random.choice(metal)
            if str(tx_list[i])=="increasing" or str(tx_list[i])=="increase":
                tx_list[i]="decreasing"
#             if str(tx_list[i])=="dog" or str(tx_list[i])=="Dogs" or str(tx_list[i])=="Dog" or str(tx_list[i])=="dogs":
#                 tx_list[i]="cat"
            tx_list[i]=WordNetLemmatizer().lemmatize(tx_list[i],'v')
            
            
        #print(tx_list)  
        new_string3=" ".join(map(str,tx_list))
        fin=[]
        tx_list1=one.split(" ")

        for i in range(len(tx_list1)):
            
            if tx_list1[i] in metal:
                tx_list1[i]=random.choice(metal)
            if str(tx_list1[i])=="increasing" or str(tx_list1[i])=="increase":
                tx_list1[i]="decreasing"
            if tx_list1[i] in number:
                tx_list1[i]=random.choice(number)
            if tx_list1[i] in day1:
                tx_list1[i]=random.choice(day1)
#             if tx_list1[i] in colors:
#                 tx_list1[i]=random.choice(colors)
            if tx_list1[i].isdigit():
                w=int(tx_list1[i])
                w=w-2
                tx_list1[i]=w
            if tx_list1[i] in yer:
                tx_list1[i]=random.choice(yer)
        new_string5=" ".join(map(str,tx_list1))
        
        fin.append(new_string5.lstrip())
        fin.append(new_string3.lstrip())
        fin.append(new_string4.lstrip())
        lr=" "
        lr=lr+'\''''
        lr=lr+'\''
        lr = '\',\''.join(map(str, fin))
        lr=lr+'\''
        f="\'"
        f=f+lr
        
#         if len(k)<2:
#             lr1=" "
#             lr1=lr1+'\''''
#             lr1=lr1+'\''
#             lr1 = '\',\''.join(map(str, single_word))
#             lr1=lr1+'\''
#             f1="\'"
#             f1=f1+lr1
#             f=f1
#             print(f)
#         else:
        print(f)
#     fin=[]
#     fin=f.split(",")
    
    
    dataset['distractor'][q]=f
        
new_dataframe = dataset[['question','answer_text','distractor']]
        #print(new_list)

'The lack of career -- based course_of_study in USA high schooling','The lack of career -- base course in US high school','The lack of career -- based courses in US high schools'


c:\users\dineshrajan\anaconda3\envs\valuelabs\lib\site-packages\ipykernel_launcher.py:332: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\dineshrajan\anaconda3\envs\valuelabs\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


'the sunlight is out at Nox','the sun be out at dark','the sun is out at night'
'have to pose confirmed at least twice','have to get confirm at least twice','have to get confirmed at least twice'
'cypher made way for him in the pee .','nobody make room for him in the water .','nobody made room for him in the water .'
'There are twelve land in the globe Wildlife stock .','There be twelve countries in the World Wildlife Fund .','There are twelve countries in the World Wildlife Fund .'
'it may clear it difficult for client to reclaim their information','it may make it difficult for customers to recover their data','it may make it difficult for customers to recover their data'
'how billy_club made blueberry juice with his uncle','how Billy make blueberry juice with his uncle','how Billy made blueberry juice with his uncle'
'The escargot "s" tooth ca "nt" make_up worn out ..','The snail 's teeth ca n't be wear out ..','The snail 's teeth ca n't be worn out ..'
'Eat the nutrient your emcee h

c:\users\dineshrajan\anaconda3\envs\valuelabs\lib\site-packages\PyDictionary\utils.py:5: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 5 of the file c:\users\dineshrajan\anaconda3\envs\valuelabs\lib\site-packages\PyDictionary\utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  return BeautifulSoup(requests.get(url).text)


'thermometer','thermometer','measuring instrument for measuring tempe'
'capital_of_Hungary .','Budapest .','Budapest .'
'departure in the learning_ability of emotional and rational hoi_polloi .','Differences in the brain of emotional and rational people .','Differences in the brains of emotional and rational people .'
'pledge the cocoa','drink the chocolate','drink the chocolate'
'Because he thinks the young homo is a bad gentleman .','Because he think the young man be a bad man .','Because he thinks the young man is a bad man .'
'A Remarkable life-time : Tortoise Dies at 248','A Remarkable Life : Tortoise Dies at 252','A Remarkable Life : Tortoise Dies at 251'
'The Shenzhou Olympics','The Shenzhou Olympics','The Shenzhou Olympics'
'ca "nt" often moisten their work_force','ca n't often wash their hand','ca n't often wash their hands'
'represent engaged in developing software_program','be engage in develop software','be engaged in developing software'
'They give it to child','They give 

...

'Most of the butt were sold to schooltime and eaten by educatee in Zhejiang responsibility .','Most of the buns be sell to school and eat by students in Zhejiang Province .','Most of the buns were sold to schools and eaten by students in Zhejiang Province .'
'scholarly_person can have_got their possess interest','students can have their own interest','students can have their own interests'
'chiropteran and their special immune arrangement','bat and their special immune system','bats and their special immune system'
'Some mass are happy to have genetic effort for their demeanour .','Some people be happy to accept genetic cause for their behavior .','Some people are happy to accept genetic causes for their behavior .'
'it can live easily operated through user "" mobile speech_sound','it can be easily operate through users ' mobile phone','it can be easily operated through users ' mobile phones'
'It encourages many old adult to bring_home_the_bacon .','It encourage many old adults to succ

'will enjoin the parents about their tiddler "s" assignment and topic per workweek','will tell the parent about their child 's assignments and subject per week','will tell the parents about their child 's assignments and subjects per week'
'Examining antecedency and deciding what is the most important to you .','Examining priorities and decide what be the most important to you .','Examining priorities and deciding what is the most important to you .'
'a popular male_monarch of soda_water in spite of ill fame .','a popular King of Pop in spite of ill fame .','a popular King of Pop in spite of ill fame .'
'with the vitality from sunlight','with the energy from sunlight','with the energy from sunlight'
'have to serve to advertise their web_site','have to help to advertise their site','have to help to advertise their site'
'A slow_down maturation for a long clip and then a point of rapid , dramatic decreasing .','A slow growth for a long time and then a period of rapid , dramatic decrease 

'both the vociferation issue and prison_term on the timekeeper','both the call number and time on the timer','both the call number and time on the timer'
'Snake_River awoke from their wintertime sopor earlier','snake awake from their Winter sleep earlier','snakes awoke from their autumn sleep earlier'
'It is "nt" Xiao Ming "s" theater .','It be n't Xiao Ming 's house .','It is n't Xiao Ming 's house .'
'only during the existence cupful','only during the World Cup','only during the World Cup'
'William_Le_Baron_Jenny "s" beginner gave her a real off-white necklace','Jenny 's father give her a real pearl necklace','Jenny 's father gave her a real pearl necklace'
'A popular necromancer from Nationalist_China','A popular magician from Taiwan','A popular magician from Taiwan'
'assure us about a midnight assistance','tell us about a midnight assistance','tell us about a midnight assistance'
'Cristoforo_Colombo never realized the full substance of his discovery','Columbus never realize the ful

'At long_time 61.when almost all had `` void nests `` .','At age 61.when almost all have " empty nest " .','At age 61.when almost all had " empty nests " .'
'secondary school_day','secondary school','secondary school'
'chinaware is trying to pack action_mechanism to figure_out such variety of job .','China be try to take action to solve such kind of problems .','China is trying to take action to solve such kind of problems .'
'it will put_up to the local economic developing','it will contribute to the local economic development','it will contribute to the local economic development'
'embody looked after by congeneric','be look after by relatives','be looked after by relatives'
'earnings aid to the mass you meet','pay attention to the people you meet','pay attention to the people you meet'
'get connected with the young child and continue active','get connect with the young kid and stay active','get connected with the young kids and stay active'
'prof . Klammer advises her scholarly_pers

'developing a loosen family_relationship with food producer','develop a loose relationship with food producers','developing a loose relationship with food producers'
'The reprocessing of the huge sum_of_money of electronic permissive_waste in the State_Department .','The reprocess of the huge amount of electronic waste in the state .','The reprocessing of the huge amounts of electronic waste in the state .'
'you still demand to give_tongue_to your salutation','you still need to express your greet','you still need to express your greetings'
'A band of wearing_apparel to tire_out together for a special social_occasion .','A set of clothe to wear together for a special occasion .','A set of clothes to wear together for a special occasion .'
'his idolatry to geographic_expedition on the Pacific ocean','his devotion to exploration on the Pacific Ocean','his devotion to exploration on the Pacific Ocean'
'often helped with the farm workplace when she visited her gran','often help with the far

In [ ]:
new_dataframe.head()

,question,answer_text,distractor
0,What 'S the main idea of the text ?,The lack of career -- based courses in US high...,'The lack of career -- based course_of_study i...
1,"In the summer high season , Finland does nt se...",the sun is out at night,"'the sunlight is out at Nox','the sun be out a..."
2,If you want to apply for Chinese Business Inte...,have to get confirmed at least twice,"'have to pose confirmed at least twice','have ..."
3,"That afternoon , the boy 's clothes were dry b...",nobody made room for him in the water .,"'cypher made way for him in the pee .','nobody..."
4,Which of the following statements is NOT true ?,There are twelve countries in the World Wildli...,'There are twelve land in the globe Wildlife s...


In [ ]:
new_dataframe.to_excel("C:\\Users\\DineshRajan\\Desktop\\DataSet\\mcq_result66.xlsx")

In [ ]:
#directions,WordNetLemmatizer().lemmatize(tx_list[i],'v') for adverbs and adjectives